<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/JTC_v3_2_level3_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependency

In [1]:
! pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 860.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=acf7035dadb2424b5fc46927f475ffe9389fa8d34e1df2440f66c3fb79874be1
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


# load packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
import json

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

from sentence_transformers import SentenceTransformer, util
from sklearn import metrics

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [4]:
from google.colab import drive
drive.mount('/content/drive')
root_path = "drive/MyDrive/trademe_data/"

Mounted at /content/drive


# load  data

In [5]:
raw_path = f"{root_path}raw.csv"

In [6]:
raw_df = pd.read_csv(raw_path)
raw_df.head(1)

,Unnamed: 0,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature
0,0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualifi asbesto remov


In [7]:
raw_df = raw_df[['y_cat_id', 'x_title_feature', 'cat_1', 'cat_2','cat_3']]
print(len(raw_df))
raw_df.head()

20000


,y_cat_id,x_title_feature,cat_1,cat_2,cat_3
0,5192,unqualifi asbesto remov,trades & services,labourers,labourers
1,5123,senior test analyst,it,testing,testing
2,6894,ict trainer supervisor,education,tutoring & training,tutoring & training
3,5197,automot specialist multi facet posit,trades & services,technicians,technicians
4,5114,busi analyst,it,business & systems analysts,business & systems analysts


# practice

In [9]:
cat_raw = pd.read_csv(f"{root_path}categories.csv")
cat_raw.head(1)

,y_cat_id,cat_1,cat_2,cat_3
0,4957,construction & roading,health & safety,health & safety


In [10]:
acc = cat_raw[cat_raw.cat_2 == "accountants"]
acc

,y_cat_id,cat_1,cat_2,cat_3
142,621,accounting,accountants,business advisory services
144,6882,accounting,accountants,assistant accountants
145,6883,accounting,accountants,audit
146,6884,accounting,accountants,financial accounting & reporting
147,6885,accounting,accountants,general / ca
148,6886,accounting,accountants,management accountants
149,6887,accounting,accountants,tax


In [11]:
cat_raw[cat_raw.cat_3 == 'tax']

,y_cat_id,cat_1,cat_2,cat_3
149,6887,accounting,accountants,tax


In [12]:
grouped = cat_raw.groupby('cat_2')

cat_2_ununique_groups = {}
for name, group in grouped:
    if len(group) > 1:
        cat_2_ununique_groups[name] = group

In [13]:
c3_grouped = cat_raw.groupby(['cat_2', 'cat_1'])

c3_dict = {}
for name, group in c3_grouped:
    # print(name)
    # print(group)
    # if len(group) > 1:
    #     print(name[0])
    #     print(group.cat_3.to_list())
    if len(group) > 1:
        c3_dict[name[0]] = group.cat_3.to_list()

In [14]:
c3_dict

{'accountants': ['business advisory services',
  'assistant accountants',
  'audit',
  'financial accounting & reporting',
  'general / ca',
  'management accountants',
  'tax'],
 'drivers & couriers': ['couriers', 'truck ht'],
 'nursing & midwifery': ['community & primary care',
  'critical care & emergency',
  'medical',
  'mental health',
  'paediatrics',
  'surgical',
  'theatre & recovery',
  'women’s health']}

# filter level 3 data

In [15]:
c2_dc_df = raw_df[raw_df.cat_2 == "drivers & couriers"]
print(c2_dc_df.cat_3.unique())
c2_dc_df

['truck ht' 'couriers']


,y_cat_id,x_title_feature,cat_1,cat_2,cat_3
36,6909,class 5 driver taupo,transport & logistics,drivers & couriers,truck ht
62,6909,class 5 driver nightshift,transport & logistics,drivers & couriers,truck ht
96,6909,storesperson,transport & logistics,drivers & couriers,truck ht
110,6909,class 5 driver,transport & logistics,drivers & couriers,truck ht
146,6909,metro driver class 5,transport & logistics,drivers & couriers,truck ht
...,...,...,...,...,...
19913,6909,hiab driver,transport & logistics,drivers & couriers,truck ht
19917,6909,driver class 4 8pm start,transport & logistics,drivers & couriers,truck ht
19922,6909,class 5 driver,transport & logistics,drivers & couriers,truck ht
19951,6909,class 5 driver levin,transport & logistics,drivers & couriers,truck ht


# encoding

In [16]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [17]:
c2_dc_df['x_title_feature_encoding'] = c2_dc_df['x_title_feature'].apply(lambda x: model.encode(str(x), convert_to_tensor=True).cpu().numpy())

<ipython-input-17-b76e54aa8ecd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c2_dc_df['x_title_feature_encoding'] = c2_dc_df['x_title_feature'].apply(lambda x: model.encode(str(x), convert_to_tensor=True).cpu().numpy())


In [18]:
c2_dc_df.head(2)

,y_cat_id,x_title_feature,cat_1,cat_2,cat_3,x_title_feature_encoding
36,6909,class 5 driver taupo,transport & logistics,drivers & couriers,truck ht,"[-0.043008037, 0.021695413, 0.011558895, 0.000..."
62,6909,class 5 driver nightshift,transport & logistics,drivers & couriers,truck ht,"[-0.0065754466, -0.0058608367, -0.0022686715, ..."


# Train & test split

In [19]:
X_train, X_test , y_train, y_test = train_test_split(c2_dc_df['x_title_feature_encoding'],   c2_dc_df[['cat_3']],  test_size=0.2,   random_state=123    )

In [20]:
X_train = X_train.to_list()
X_test = X_test.to_list()

# Classification

In [21]:
from typing import Any, Optional

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
import pickle

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [22]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

<ipython-input-22-549420f83866>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestClassifier()

In [23]:
preds = clf.predict(X_test)

In [24]:
from datetime import datetime
start = datetime.utcnow()

t = clf.predict([X_test[0]])

date= datetime.utcnow() - start
print("Number of days since epoch:",date)
seconds =(date.total_seconds())
milliseconds = round(seconds*1000)
print("Milliseconds since epoch:",milliseconds)

Number of days since epoch: 0:00:00.018009
Milliseconds since epoch: 18


In [25]:
probas = clf.predict_proba(X_test)

In [26]:
report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

    couriers       1.00      0.64      0.78        25
    truck ht       0.93      1.00      0.96       117

    accuracy                           0.94       142
   macro avg       0.96      0.82      0.87       142
weighted avg       0.94      0.94      0.93       142

